In [1]:
import tensorflow as tf
import utils
import numpy as np

__author__ = "Olivares Castillo José Luis"
tf.__version__


'1.5.0-rc0'

In [2]:
es,na = utils.load_node2vec()

In [3]:
es.shape[1]

129

In [4]:
index_es, index_na = utils.get_seed_index(es, na)
es_vectores = utils.get_vectors(es, index_es)
na_vectores = utils.get_vectors(na, index_na)

In [5]:
#index_es

In [6]:
#es[es[0] == "hongo"][0]

In [7]:
es_dummy = es.drop(es.columns[0],axis=1)
na_dummy = na.drop(na.columns[0],axis=1)
es_vectores1 = np.array(es_dummy)
na_vectores1 = np.array(na_dummy)
print(na_vectores1.shape)

(1904, 128)


In [8]:
#es_vectores = [es.iloc[_].loc[1::] for _ in range(es.shape[0])]

In [9]:
# Palabra y vector
indice = 1756
print(es.iloc[indice][0])
test_vectors = np.array(es.iloc[indice][1::]).astype(np.float)
test_vectors.resize(1,128)
print(test_vectors.shape)

4
(1, 128)


In [10]:
sess = tf.Session()

In [11]:
saver = tf.train.import_meta_graph('./models/Adagrad_H_305_LR_0.433.ckpt.meta')
saver.restore(sess,tf.train.latest_checkpoint('./models/'))

INFO:tensorflow:Restoring parameters from ./models/Adagrad_H_305_LR_0.433.ckpt


In [12]:
graph = tf.get_default_graph()

In [13]:
#print(sess.run("output/W:0"))

In [14]:
output_W = graph.get_tensor_by_name("output/W:0")
output_b = graph.get_tensor_by_name("output/b:0")
output_xwb = graph.get_tensor_by_name("output/xw_plus_b:0")

In [15]:
X = graph.get_tensor_by_name("input/input_es:0")
y = graph.get_tensor_by_name("input/target_na:0")

In [16]:
print(X.shape)

(?, 128)


In [17]:
feed_dict = {X:test_vectors}

In [18]:
output_NN = graph.get_tensor_by_name("output_1:0")

In [19]:
pred = sess.run(output_NN,feed_dict)
#print (type(pred[0]),pred[0])

In [20]:
#print(pred[0].shape)

In [21]:
top_10=utils.get_top10_closest(pred[0],na_vectores1)

In [22]:
top_10

[(1414, 0.32692518075018884),
 (1644, 0.32934493270254606),
 (1357, 0.33352179104412194),
 (1581, 0.33428527784134665),
 (1740, 0.33601810498310458),
 (914, 0.3367677098705526),
 (689, 0.33755189319903101),
 (1380, 0.33911250609107657),
 (410, 0.33941662748797408),
 (1390, 0.34190079268815976)]

In [23]:
closest = utils.get_closest_words_to(top_10,na)

In [24]:
print(es.iloc[indice][0])

4


In [25]:
closest

['diego',
 'dral',
 'federal',
 'club',
 'secretario',
 'castillo',
 'acamapich',
 'anechhuall',
 'organil',
 'rosario']